In [73]:
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

In [74]:
# 문장 추출
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.okt = Okt()
        
        self.stopwords =['이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신', '안', '어떤', '내', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', '여자', '개', '전', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '소리', '놓']
    
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)      
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences

    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
                nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence)) 
                                       if noun not in self.stopwords and len(noun) > 1]))
        return nouns

In [75]:
# TF-IDF 모델 생성 및 그래프 생성
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

In [76]:
#TextRank 알고리즘 적용
class Rank(object):
    def get_ranks(self, graph, d=0.85): 
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 
            link_sum = np.sum(A[:,id])
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) 
        return {idx: r[0] for idx, r in enumerate(ranks)}

In [77]:
#TextRank Class 구현
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        
        self.sentences = self.sent_tokenize.text2sentences(text)
        
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)

        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)

        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        index.sort()

        for idx in index:
            summary.append(self.sentences[idx])
        return summary

    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)

        keywords = []
        index=[]
        
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)

        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])

        return keywords

In [78]:
text = '''
동지중해 천연자원 개발을 두고 그리스와 대치 중인 터키의 레제프 타이이프 에르도안 대통령이 키리아코스 미초타키스 그리스 총리와 대화 의사를 밝혔다.
에르도안 대통령은 19일(현지시간) 기자들과 만난 자리에서 "선의가 있다면 미초타키스 총리와 만날 수 있다"며 "화상 회담을 하거나 제3국에서 회담할 수 있을 것"이라고 말했다.
이어 "터키는 대화 테이블에서 벗어나지 않을 것"이라며 "우리가 어떤 틀에서 무엇을 논의할지가 핵심"이라고 덧붙였다. 에르도안 대통령은 동지중해에서 천연가스 탐사 임무를 수행하던 지질 조사선 '오루츠 레이스'가 터키 남부 안탈리아 연안으로 귀환한 것을 언급하면서 "외교에 기회를 주자"고 말했다.
'''
textrank = TextRank(text)
txtComment = []

num = 1;

for row in textrank.summarize(3):
    txtComment.append(row)
    
for sentence in txtComment:
    print(str(num) + "." + sentence)
    num = num + 1
    
print("\n상위 5개 키워드\n")
for key in textrank.keywords()[:5]:
    print(key)

1.동 지중해 천연자원 개발을 두고 그리스와 대치 중인 터키의 레제 프 타이 이 프 에 르도 안 대통령이 키 리아 코스 미 초 타 키스 그리스 총리와 대화 의사를 밝혔다.
2.이어 " 터키는 대화 테이블에서 벗어나지 않을 것" 이라며 " 우리가 어떤 틀에서 무엇을 논의할 지가 핵심" 이라고 덧붙였다.
3.에 르도 안 대통령은 동 지중해에서 천연가스 탐사 임무를 수행하던 지질 조사선 ' 오루 츠 레이스' 가 터키 남부 안 탈리아 연안으로 귀환한 것을 언급하면서 " 외교에 기회를 주자" 고 말했다.

상위 5개 키워드

대통령
터키
지중해
총리와
키스
